<a href="https://colab.research.google.com/github/matherdy/Ukraine-War-Reserach/blob/master/Research_Energy_Stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 KB 713.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 KB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 4.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting uninstall: html5lib
    Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
  

In [ ]:
#pip install yfinance
#pip install scikit-learn
#pip install tweepy
#pip install mariadb

In [ ]:
import yfinance as yf
import plotly.graph_objs as go
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")



Mounted at /content/drive/


In [ ]:
data = yf.download(tickers='ICLN', period='1y', interval='1d',prepost= True)

[*********************100%***********************]  1 of 1 completed


In [ ]:
def bollinger_band(data):
    window = 20
    rolling_mean = data['Close'].rolling(window).mean()
    rolling_std = data['Close'].rolling(window).std()
    
    data['Middle Band'] = rolling_mean
    data['Short Upper Band'] = data['Middle Band'] + 1.5*rolling_std  # Calculates 10 day moving average for bands
    data['Short Lower Band'] = data['Middle Band'] - 1.5*rolling_std
    
    data['Medium Upper Band'] = data['Middle Band'] + 2*rolling_std  # Calculates 20 day moving average for bands
    data['Medium Lower Band'] = data['Middle Band'] - 2*rolling_std
    
    data['Long Upper Band'] = data['Middle Band'] + 2.5*rolling_std  # Calculates 50 day moving average for bands
    data['Long Lower Band'] = data['Middle Band'] - 2.5*rolling_std
    
    
bollinger_band(data)



In [ ]:
"""
RSI and Bollinger Band:
https://www.analyticsvidhya.com/blog/2021/01/algotrading-using-technical-indicator-and-ml-models/
"""

def rsi(data, window = 14):
    close = data["Close"]
    
    #Price increase or decrease over previous day
    dif = close.diff()
    dif = dif[1:]
    
    # pos_m identifies stock price going up
    # neg_m identifies stock price going down
    pos_m, neg_m = dif.copy(), dif.copy()
    pos_m[pos_m < 0] = 0
    neg_m[neg_m > 0] = 0
    
    # Positive Rolling Mean Exponential
    prme = pos_m.ewm(span=window).mean()
    # Negative Rolling mean Exponential 
    nrme = neg_m.abs().ewm(span=window).mean()
    
    # Ratio of magnitude of up move to down move
    RSE = prme / nrme
    RSIE = 100 - (100/(1+RSE))
    data["RSIE"] = RSIE
    
    # Positive Rolling Mean Simple
    prms = pos_m.rolling(window).mean()
    # Negative Rolling Mean Simple
    nrms = neg_m.rolling(window).mean()
    
    RSS = prms / nrms
    RSIS = 100.0 - (100.0 / (1.0 + RSS))
    
    data['rsis'] = RSIS
    return data

rsi(data)

,Open,High,Low,Close,Adj Close,Volume,Middle Band,Short Upper Band,Short Lower Band,Medium Upper Band,Medium Lower Band,Long Upper Band,Long Lower Band,RSIE,rsis
Date,,,,,,,,,,,,,,,
2022-02-17,18.629999,18.770000,18.410000,18.480000,18.322655,6010400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-02-18,18.420000,18.500000,18.040001,18.139999,17.985550,4887800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
2022-02-22,17.860001,17.990000,17.549999,17.690001,17.539383,6579800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
2022-02-23,18.000000,18.059999,17.520000,17.570000,17.420403,4631700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
2022-02-24,17.209999,19.010000,17.200001,18.910000,18.748995,14243500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.888721,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-13,19.709999,20.000000,19.600000,19.870001,19.870001,2154400,20.4870,21.181040,19.792959,21.412387,19.561613,21.643734,19.330266,37.222175,-104.255626
2023-02-14,19.650000,19.820000,19.410000,19.730000,19.730000,3689300,20.4055,21.071510,19.739490,21.293513,19.517487,21.515516,19.295483,33.693313,-107.692485
2023-02-15,19.540001,20.160000,19.540001,20.150000,20.150000,2342200,20.3575,20.979648,19.735352,21.187030,19.527969,21.394413,19.320586,50.076634,-291.667263


In [ ]:
#declare figure
fig = go.Figure()



fig.add_trace(go.Scatter(x=data.index, y= data['Middle Band'],line=dict(color='blue', width=.7), name = 'Middle Band'))

fig.add_trace(go.Scatter(x=data.index, y= data['Short Upper Band'],line=dict(color='pink', width=1.5), name = 'Short Upper Band (Sell)'))
fig.add_trace(go.Scatter(x=data.index, y= data['Short Lower Band'],line=dict(color='lightgreen', width=1.5), name = 'Short Lower Band (Buy)'))

fig.add_trace(go.Scatter(x=data.index, y= data['Medium Upper Band'],line=dict(color='red', width=1.5), name = 'Medium Upper Band (Sell)'))
fig.add_trace(go.Scatter(x=data.index, y= data['Medium Lower Band'],line=dict(color='green', width=1.5), name = 'Medium Lower Band (Buy)'))

fig.add_trace(go.Scatter(x=data.index, y= data['Long Upper Band'],line=dict(color='darkred', width=1.5), name = 'Long Upper Band (Sell)'))
fig.add_trace(go.Scatter(x=data.index, y= data['Long Lower Band'],line=dict(color='darkgreen', width=1.5), name = 'Long Lower Band (Buy)'))
    
    



In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=data.index, y= data['Middle Band'],line=dict(color='blue', width=.7), name = 'Middle Band'))

fig.add_trace(go.Scatter(x=data.index, y= data['Short Upper Band'],line=dict(color='pink', width=1.5), name = 'Short Upper Band (Sell)'))
fig.add_trace(go.Scatter(x=data.index, y= data['Medium Upper Band'],line=dict(color='red', width=1.5), name = 'Medium Upper Band (Sell)'))
fig.add_trace(go.Scatter(x=data.index, y= data['Long Upper Band'],line=dict(color='darkred', width=1.5), name = 'Long Upper Band (Sell)'))
fig.add_trace(go.Scatter(x=data.index, y= data['Short Lower Band'],line=dict(color='lightgreen', width=1.5), name = 'Short Lower Band (Buy)'))
fig.add_trace(go.Scatter(x=data.index, y= data['Medium Lower Band'],line=dict(color='green', width=1.5), name = 'Medium Lower Band (Buy)'))
fig.add_trace(go.Scatter(x=data.index, y= data['Long Lower Band'],line=dict(color='darkgreen', width=1.5), name = 'Long Lower Band (Buy)'))

#Candlestick
fig.add_trace(go.Candlestick(x=data.index,
                open=data['Open'],
                high=data['High'],
                low=data['Low'],
                close=data['Close'], name = 'market data'))

# Add titles
fig.update_layout(
    title='ICLN live share price evolution',
    yaxis_title='Stock Price (USD per Shares)')

# X-Axes
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=15, label="15m", step="minute", stepmode="backward"),
            dict(count=45, label="45m", step="minute", stepmode="backward"),
            dict(count=1, label="HTD", step="hour", stepmode="todate"),
            dict(count=3, label="3h", step="hour", stepmode="backward"),
            dict(step="all")
        ])
    )
)



In [ ]:
data.dropna(inplace=True)
data["BB_Signal"] = 0

data["BB_Signal"] = np.where(data["Low"] < data["Medium Lower Band"], 1, data["BB_Signal"]) # buy signal

data["BB_Signal"] = np.where(data["Close"] > data["Medium Upper Band"], -1, data["BB_Signal"]) # sell signal



In [ ]:
test = data[data["BB_Signal"] !=0]

In [ ]:
fig.add_trace(go.Scatter(x=data.index[data["BB_Signal"]==1], y = data["Close"][data["BB_Signal"]==1], 
                         mode = "markers", marker_color = "darkgreen", marker_symbol = "arrow-up", marker_size = 10,
                        name = "Buy Signal")) #buy signal
fig.add_trace(go.Scatter(x=data.index[data["BB_Signal"]==-1], y = data["Close"][data["BB_Signal"]==-1], 
                         mode = "markers", marker_color = "darkred", marker_symbol = "arrow-down", marker_size = 10,
                        name = "Sell Signal")) #sell signal

In [ ]:
def get_returns(data):
    entrie_prices = []
    exit_prices = []
    entrie_dates = []
    exit_dates = []
    in_market = False
    for i in range(data.shape[0]):
        if (in_market == False) and (data.iloc[i]["BB_Signal"] == 1):
            in_market = True
            entrie_prices.append((data.iloc[i]["Close"]))
            entrie_dates.append(data.iloc[i].name)
        if in_market == True and (data.iloc[i]["BB_Signal"] == -1):
            in_market = False
            exit_prices.append((data.iloc[i]["Close"]))
            exit_dates.append(data.iloc[i].name)
    if len(entrie_prices) > len(exit_prices):
        exit_prices.append(data.iloc[-1]["Close"])
        exit_dates.append(data.iloc[-1].name)
    return sum(exit_prices)-sum(entrie_prices),entrie_dates, exit_dates

returns, entries, exits = get_returns(data)

In [ ]:
entries, exits, returns

([Timestamp('2022-04-21 00:00:00'),
  Timestamp('2022-09-01 00:00:00'),
  Timestamp('2022-12-20 00:00:00'),
  Timestamp('2023-02-07 00:00:00')],
 [Timestamp('2022-07-27 00:00:00'),
  Timestamp('2022-11-10 00:00:00'),
  Timestamp('2023-01-12 00:00:00'),
  Timestamp('2023-02-17 00:00:00')],
 0.0)